# Tutorial: Leakage scheme implementation

## Author: Leo Werneck

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#initialize_nrpy): Initialize NRPy+/Python modules
1. [Step 2](#declare_c_params): Declare C parameters
1. [Step 3](#fermi_dirac_integrals): Fermi-Dirac integrals
1. [Step 4](#free_emission_cooling_rates): Free neutrino emission and cooling rates
    1. [Step 4.a](#beta_processes): Electron and positron capture ($\beta$-processes)
        1. [Step 4.a.i](#degeneracy_parameters): Compute all degeneracy parameters
        1. [Step 4.a.ii](#energy_moments): Compute energy-moments
        1. [Step 4.a.iii](#beta_blocking_factors): Blocking factors
        1. [Step 4.a.iv](#beta_emission_rates): Free neutrino emission rates
    1. [Step 4.b](#pair_annihilation): Electron-positron pair annihilation
    1. [Step 4.c](#plasmon_decay): Transverse plasmon decay
    1. [Step 4.d](#bremsstrahlung): Nucleon-nucleon Bremsstrahlung
    1. [Step 4.e](#free_cooling_rates): Free neutrino cooling rates
    1. [Step 4.f](#total_rates): Total emission and cooling rates for free neutrinos
1. [Step 5](#grmhd_source_terms): GRMHD source terms
    1. [Step 5.a](#neutrino_number_and_energy_densities): Neutrino number and energy densities
    1. [Step 5.b](#direct_loss_time_scales): Direct neutrino loss time scales
    1. [Step 5.c](#scattering_opacities): Scattering opacities
    1. [Step 5.d](#absorption_opacities): Absorption opacities
    1. [Step 5.e](#total_transport_opacities): Total transport opacities
    1. [Step 5.f](#diffusion_timescales): Diffusion timescales
    1. [Step 5.g](#effective_emission_rates): Effective emission rates
    1. [Step 5.h](#grmhd_source_terms_impl): Computing $\mathcal{R}$ and $\mathcal{Q}$
    1. [Step 5.i](#grmhd_source_terms_c_function_generation): C function generation
        1. [Step 5.i.i](#low_level_functions): Low level functions
        1. [Step 5.i.ii](#driver_function): Driver function
1. [Step 6](#optical_depth): Optical depth
1. [Step 7](#writing_leakage_hh): Writing the C header file `Leakage.h`
1. [Step 8](#c_code_generation): C code generation

<a id='initialize_nrpy'></a>

# Step 1: Initialize NRPy+/Python modules \[Back to [Top](#toc)\]
$$\label{initialize_nrpy}$$

In [1]:
import os,sys,shutil
import sympy as sp
import astropy.constants as ct
from collections import namedtuple
sys.path.append("nrpy_core")
import outputC as outC
import NRPy_param_funcs as par
import cmdline_helper as cmd
import optical_depth_helpers as tau_helpers

generate_ET_thorn = True
debug_mode = False

if generate_ET_thorn:
    Thorndir  = "NRPyLeakageET"
    shutil.rmtree(Thorndir,ignore_errors=True)
    cmd.mkdir(Thorndir)
    Ccodesdir = os.path.join(Thorndir,"src")
else:
    Ccodesdir = os.path.join("standalone","NRPyLeakage")
shutil.rmtree(Ccodesdir,ignore_errors=True)
cmd.mkdir(Ccodesdir)

function_prototypes = []
make_code_defn_list = []

def adjust_params_indent(c_type,name,params_in):

    params = params_in.replace('\n','')
    params = params.split(",")

    # Remove all white space in the beginning
    # and the end of the parameters
    N_params = len(params)
    for i in range(N_params):
        while params[i][0] == " ":
            params[i] = params[i][1:]
        while params[i][-1] == " ":
            params[i] = params[i][:-1]
    
    # Begin indentation with two spaces: one for
    # the space between the c type and function name
    # and one for the parenthesis of the function arguments
    indent = "  "
    
    # Now increment indent based on the function name
    # and its c type
    for i in range(len(c_type)+len(name)):
        indent += " "
        
    # Now write the new parameter string
    new_params = params[0]+",\n"
    for i in range(1,N_params-1):
        new_params += indent+params[i]+",\n"
    new_params += indent+params[-1]
    
    return new_params

<a id='declare_c_params'></a>

# Step 2: Declare C parameters \[Back to [Top](#toc)\]
$$\label{declare_c_params}$$

We start by implementing a series of constants that will be used by our C code. These appear in the various equations in appendices A and B of [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R). For convenience, we use the [Astropy](https://www.astropy.org/) package to fetch the values of physical constants and for unit conversion.

In [2]:
# C parameters
thismodule = "NRPyLeakage"

c  = ct.c.cgs                    # cm / s
G  = ct.G.cgs                    # cm^3 / g / s^2
M  = ct.M_sun.cgs                # g
L  = G * M / c**2                # cm
T  = L/c                         # s
h   = ct.h.cgs                   # erg s

D = M/L**3                       # g / cm^3
R = D/T                          # g / cm^3 / s
Q = (c**2*D/T).to("MeV/(cm3 s)") # g / cm / s^3
hc3 = (h*c)**3                   # erg^3 cm^3

# These parameters enable or disable certain interactions
enable_beta_nue         = par.Cparameters("#define",thismodule,thismodule+"_enable_beta_nue"        ,"1")
enable_beta_anue        = par.Cparameters("#define",thismodule,thismodule+"_enable_beta_anue"       ,"1")
enable_pair_nue_anue    = par.Cparameters("#define",thismodule,thismodule+"_enable_pair_nue_anue"   ,"1")
enable_pair_nux_anux    = par.Cparameters("#define",thismodule,thismodule+"_enable_pair_nux_anux"   ,"1")
enable_plasmon_nue_anue = par.Cparameters("#define",thismodule,thismodule+"_enable_plasmon_nue_anue","1")
enable_plasmon_nux_anux = par.Cparameters("#define",thismodule,thismodule+"_enable_plasmon_nux_anux","1")
enable_brems_nui_anui   = par.Cparameters("#define",thismodule,thismodule+"_enable_brems_nui_anui"  ,"1")

# These parameters allow us to select which constants to use in the C code
par.Cparameters("#define",thismodule,"USE_NRPY_CONSTANTS"   ,"0")
par.Cparameters("#define",thismodule,"USE_HARM_CONSTANTS"   ,"1")
par.Cparameters("#define",thismodule,"USE_ZELMANI_CONSTANTS","2")

# NRPy parameters 
NRPy_Q_npmass   = par.Cparameters("#define",thismodule,thismodule+"_Q_npmass"           ,"1.2935")
NRPy_gamma_0    = par.Cparameters("#define",thismodule,thismodule+"_gamma_0"            ,"5.565e-2")
NRPy_sigma_0    = par.Cparameters("#define",thismodule,thismodule+"_sigma_0"            ,"1.76e-44")
NRPy_alpha      = par.Cparameters("#define",thismodule,thismodule+"_alpha"              ,"1.25")
NRPy_C_A        = par.Cparameters("#define",thismodule,thismodule+"_C_A"                ,"0.5")
NRPy_sinthw2    = par.Cparameters("#define",thismodule,thismodule+"_sinthw2"            ,"0.23")
NRPy_Brems_C1   = par.Cparameters("#define",thismodule,thismodule+"_Brems_C1"           ,"2.9988e7")
NRPy_Brems_C2   = par.Cparameters("#define",thismodule,thismodule+"_Brems_C2"           ,"6.5428e7")
NRPy_Brems_zeta = par.Cparameters("#define",thismodule,thismodule+"_Brems_zeta"         ,"0.5")
NRPy_eta_nue_0  = par.Cparameters("#define",thismodule,thismodule+"_eta_nue_0"          ,"0.0")
NRPy_eta_anue_0 = par.Cparameters("#define",thismodule,thismodule+"_eta_anue_0"         ,"0.0")
NRPy_c_light    = par.Cparameters("#define",thismodule,thismodule+"_c_light"            ,"%.15e"%c.value)
NRPy_N_A        = par.Cparameters("#define",thismodule,thismodule+"_N_A"                ,"%.15e"%ct.N_A.cgs.value)
NRPy_alpha_fs   = par.Cparameters("#define",thismodule,thismodule+"_alpha_fs"           ,"%.15e"%ct.alpha.cgs.value)
NRPy_amu        = par.Cparameters("#define",thismodule,thismodule+"_amu"                ,"%.15e"%ct.u.cgs.value)
NRPy_hc3        = par.Cparameters("#define",thismodule,thismodule+"_hc3"                ,"%.15e"%hc3.value)
NRPy_m_e_c2     = par.Cparameters("#define",thismodule,thismodule+"_m_e_c2"             ,"%.15e"%(ct.m_e.cgs*c**2).to("MeV").value)
NRPy_Dgeomtocgs = par.Cparameters("#define",thismodule,thismodule+"_units_geom_to_cgs_D","%.15e"%D.value)
NRPy_Dcgstogeom = par.Cparameters("#define",thismodule,thismodule+"_units_cgs_to_geom_D","%.15e"%(1.0/D.value))
NRPy_Rcgstogeom = par.Cparameters("#define",thismodule,thismodule+"_units_cgs_to_geom_R","%.15e"%(1.0/R.value))
NRPy_Qcgstogeom = par.Cparameters("#define",thismodule,thismodule+"_units_cgs_to_geom_Q","%.15e"%(1.0/Q.value))
NRPy_Mgeomtocgs = par.Cparameters("#define",thismodule,thismodule+"_units_geom_to_cgs_M","%.15e"%M.value)
NRPy_Lgeomtocgs = par.Cparameters("#define",thismodule,thismodule+"_units_geom_to_cgs_L","%.15e"%L.value)
NRPy_Tgeomtocgs = par.Cparameters("#define",thismodule,thismodule+"_units_geom_to_cgs_T","%.15e"%T.value)
NRPy_Mcgstogeom = par.Cparameters("#define",thismodule,thismodule+"_units_cgs_to_geom_M","%.15e"%(1.0/M.value))
NRPy_Lcgstogeom = par.Cparameters("#define",thismodule,thismodule+"_units_cgs_to_geom_L","%.15e"%(1.0/L.value))
NRPy_Tcgstogeom = par.Cparameters("#define",thismodule,thismodule+"_units_cgs_to_geom_T","%.15e"%(1.0/T.value))

# HARM parameters
harm_c_light = par.Cparameters("#define",thismodule,thismodule+"_harm_c_light"            ,"2.99792458e10")
par.Cparameters("#define",thismodule,thismodule+"_harm_N_A"                ,"6.0221415e23")
par.Cparameters("#define",thismodule,thismodule+"_harm_alpha_fs"           ,"0.00729735252051")
par.Cparameters("#define",thismodule,thismodule+"_harm_amu"                ,"1.66053886e-24")
par.Cparameters("#define",thismodule,thismodule+"_harm_hc3"                ,"1.90589514992e-30")
harm_m_e_c2 = par.Cparameters("#define",thismodule,thismodule+"_harm_m_e_c2"             ,"5.109989461e-01")
par.Cparameters("#define",thismodule,thismodule+"_harm_units_geom_to_cgs_D","6.17714470405638e17")
par.Cparameters("#define",thismodule,thismodule+"_harm_units_cgs_to_geom_D","1.61887093132742e-18")
par.Cparameters("#define",thismodule,thismodule+"_harm_units_cgs_to_geom_R","7.97315453692e-24")
par.Cparameters("#define",thismodule,thismodule+"_harm_units_cgs_to_geom_Q","1.42134688491e-50")
par.Cparameters("#define",thismodule,thismodule+"_harm_units_geom_to_cgs_L","%.15e"%L.value)

# ZelmaniLeak parameters
ZL_Q_npmass = par.Cparameters("#define",thismodule,thismodule+"_ZL_Q_npmass","1.293333")
ZL_alpha    = par.Cparameters("#define",thismodule,thismodule+"_ZL_alpha"   ,"1.23")
ZL_N_A      = par.Cparameters("#define",thismodule,thismodule+"_ZL_N_A"     ,"6.0221367e+23")
ZL_alpha_fs = par.Cparameters("#define",thismodule,thismodule+"_ZL_alpha_fs","7.297352520505561e-03")
ZL_amu      = par.Cparameters("#define",thismodule,thismodule+"_ZL_amu"     ,"1.674927211e-24")
ZL_hc3      = par.Cparameters("#define",thismodule,thismodule+"_ZL_hc3"     ,"1.905893979207552e-30")
ZL_m_e_c2   = par.Cparameters("#define",thismodule,thismodule+"_ZL_m_e_c2"  ,"5.1099891e-01")

<a id='fermi_dirac_integrals'></a>

# Step 3: Fermi-Dirac integrals \[Back to [Top](#toc)\]
$$\label{fermi_dirac_integrals}$$

Below we implement a function to evaluate Fermi-Dirac integrals

$$
F_{N}(z) = \int_{0}^{\infty}dx\frac{x^{N}}{e^{z-x}+1}\; ,
$$

according to the approximations in [Takahashi, Eid, and Hillebrandt (1978)](https://adsabs.harvard.edu/pdf/1978A%26A....67..185T).

In [3]:
# For N=0, we have an analytic expression
x,z  = sp.symbols('x z',real=True)
one  = sp.sympify(1)
expz = sp.exp(z)
F_0  = sp.simplify(sp.integrate(1.0/(sp.exp(x-z)+1),(x,0,sp.oo)))

# N = 0 case
F_zlarge = [F_0]
F_zsmall = [F_0]

# N=1 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,2)*z**2 + sp.sympify(1.6449))/(one+sp.exp(-sp.sympify(1.6855)*z)))
# z < 1e-3
F_zsmall.append(expz/( one + sp.sympify(0.2159)*sp.exp(sp.sympify(0.8857)*z) ))

# N=2 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,3)*z**3 + sp.sympify(3.2899)*z)/(one - sp.exp(-sp.sympify(1.8246)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(2)*expz/( one + sp.sympify(0.1092)*sp.exp(sp.sympify(0.8908)*z) ))

# N=3 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,4)*z**4 + sp.sympify(4.9348)*z**2 + sp.sympify(11.3644))/(one+sp.exp(-sp.sympify(1.9039)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(6)*expz/( one + sp.sympify(0.0559)*sp.exp(sp.sympify(0.9069)*z) ))

# N=4 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,5)*z**5 + sp.sympify(6.5797)*z**3+sp.sympify(45.4576)*z)/(one-sp.exp(-sp.sympify(1.9484)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(24)*expz/( one + sp.sympify(0.0287)*sp.exp(sp.sympify(0.9257)*z) ))

# N=5 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,6)*z**6 + sp.sympify(8.2247)*z**4 + sp.sympify(113.6439)*z**2 + sp.sympify(236.5323))/(one+sp.exp(-sp.sympify(1.9727)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(120)*expz/( one + sp.sympify(0.0147)*sp.exp(sp.sympify(0.9431)*z)))

def Cfunc_NRPyLeakage_Fermi_Dirac_Integrals():
    desc = """(c) Leo Werneck
Compute Fermi-Dirac integrals according to the approximations
in Takahashi, Eid, and Hillebrandt (1978)
https://adsabs.harvard.edu/pdf/1978A%26A....67..185T"""
    if generate_ET_thorn:
        includes = ["NRPyLeakage.h"]
    else:
        includes = ["Basic_defines.h"]
    c_type = "REAL"
    name   = "NRPyLeakage_Fermi_Dirac_integrals"
    params = "const int k, const REAL z"
    outCparams = "outCverbose=False,includebraces=False"
    prefunc = ""
    zlarge_body = ""
    zsmall_body = ""
    for k in range(6):
        zlarge_body += "    case("+str(k)+"):\n"
        zlarge_body += "      "+outC.outputC(F_zlarge[k],"Fermi_Dirac_integral","returnstring",params=outCparams)
        zsmall_body += "    case("+str(k)+"):\n"
        zsmall_body += "      "+outC.outputC(F_zsmall[k],"Fermi_Dirac_integral","returnstring",params=outCparams)
        if k < 5:
            zlarge_body += "      break;\n"
            zsmall_body += "      break;\n"
        else:
            zlarge_body += "      break;"
            zsmall_body += "      break;"
    body   = """
  REAL Fermi_Dirac_integral = 0.0;
  if(z>1e-3) {
    switch(k) {
"""+zlarge_body+r"""
    default:
      fprintf(stderr,"Unsuported value of k: %d\n",k);
      exit(1);
    }
  }
  else {
    switch(k) {
"""+zsmall_body+r"""
    default:
      fprintf(stderr,"Unsuported value of k: %d\n",k);
      exit(1);
    }
  }
  
  return Fermi_Dirac_integral;
"""
    loopopts = ""
    outC.outCfunction(os.path.join(Ccodesdir,name+".c"),
                      includes=includes,desc=desc,prefunc=prefunc,c_type=c_type,name=name,
                      params=params,body=body,loopopts=loopopts,enableCparameters=False)

    global function_prototypes
    function_prototypes.append(c_type+" "+name+"("+params+");\n")
    function_prototypes = outC.superfast_uniq(function_prototypes)
    if generate_ET_thorn:
        global make_code_defn_list
        make_code_defn_list.append(name+".c")

def Fermi_Dirac_validation(k,z):
    if k > 5:
        print("Unsupported value of k")
        sys.exit(1)  
    if z > 1e-3:
        F = sp.lambdify(sp.symbols('z',real=True),F_zlarge[k])
    else:
        F = sp.lambdify(sp.symbols('z',real=True),F_zsmall[k])
    return F(z)

import fdint
for k in range(6):
    print("Order %d: %e"%(k,(1.0-fdint.fdk(k,1e-5)/Fermi_Dirac_validation(k,1e-5))))
    print("Order %d: %e"%(k,(1.0-fdint.fdk(k,1e-1)/Fermi_Dirac_validation(k,1e-1))))

Order 0: 0.000000e+00
Order 0: 1.110223e-16
Order 1: -3.766628e-05
Order 1: -1.662731e-05
Order 2: 8.862389e-06
Order 2: -5.354045e-05
Order 3: 2.803556e-05
Order 3: 2.572759e-06
Order 4: -1.960806e-05
Order 4: -2.903726e-05
Order 5: -3.869268e-05
Order 5: -3.966739e-07


<a id='free_emission_cooling_rates'></a>

# Step 4: Free neutrino emission and cooling rates \[Back to [Top](#toc)\]
$$\label{free_emission_cooling_rates}$$

<a id='beta_processes'></a>

## Step 4.a: Electron and positron capture ($\beta$-processes) \[Back to [Top](#toc)\]
$$\label{beta_processes}$$

For electrons captured by protons, we use (cf. Eqs. B1 and B2 in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R))

$$
\newcommand{\mue}{\mu_{\rm e}}
\newcommand{\mun}{\mu_{\rm n}}
\newcommand{\mup}{\mu_{\rm p}}
\newcommand{\muhat}{\hat{\mu}}
\newcommand{\me}{m_{\rm e}}
\newcommand{\nb}{n_{\rm b}}
\newcommand{\ee}{e^{-}}
\newcommand{\ae}{e^{+}}
\newcommand{\nue}{\nu_{\rm e}}
\newcommand{\anue}{\bar{\nu}_{\rm e}}
\newcommand{\nui}{\nu_{\rm i}}
\newcommand{\anui}{\bar{\nu}_{\rm i}}
\newcommand{\nux}{\nu_{\rm x}}
\newcommand{\anux}{\bar{\nu}_{\rm x}}
\newcommand{\rhob}{\rho_{\rm b}}
\newcommand{\ye}{Y_{\rm e}}
\newcommand{\QQ}{\mathcal{Q}}
\newcommand{\RR}{\mathcal{R}}
\newcommand{\etae}{\eta_{\rm e}}
\newcommand{\etap}{\eta_{\rm p}}
\newcommand{\etan}{\eta_{\rm n}}
\newcommand{\etanue}{\eta_{\nue}}
\newcommand{\etaanue}{\eta_{\anue}}
\newcommand{\etanux}{\eta_{\nux}}
\newcommand{\etanui}{\eta_{\nui}}
\newcommand{\dif}{\mathrm{d}}
\newcommand{\rate}[3]{#1_{#3}^{#2}}
\newcommand{\etahat}{\hat{\eta}}
\begin{align}
  \rate{\RR}{\nue}{\rm ec} &= \frac{1+3\alpha^{2}}{8}\beta \,N_{A}\rhob Y_{\rm pn}\varepsilon_{4}(+\etae)\langle1-f_{\nue}\rangle_{\rm ec}\;,\\
  %%%%
  \rate{\RR}{\anue}{\rm pc} &= \frac{1+3\alpha^{2}}{8}\beta\,N_{A}\rhob Y_{\rm np}\varepsilon_{4}(-\etae)\langle1-f_{\anue}\rangle_{\rm pc}\;,
\end{align}
$$

where $N_{A}$ is Avogadro's number and we have the blocking factors (cf. Eqs. B3 and B4 in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R))
$$
\begin{align}
\langle1-f_{\nue}\rangle_{\rm ec} &= \left\{1 + \exp\left[\eta_{\nue}-\frac{F_{5}(\eta_{\rm e})}{F_{4}(\eta_{\rm e})}\right]\right\}^{-1}\; ,\\
\langle1-f_{\nue}\rangle_{\rm pc} &= \left\{1 + \exp\left[\eta_{\anue}-\frac{F_{5}(-\eta_{\rm e})}{F_{4}(-\eta_{\rm e})}\right]\right\}^{-1}\; ,
\end{align}
$$

where $\etae = \mue/T$ is the electron degeneracy parameter, $\mue$ is the electron chemical potential, and $T$ is the temperature. We have also defined the constant

$$
\beta = \frac{\sigma_{0}c}{(\me c^{2})^{2}}\;,
$$

where $\me$ is the electron mass, $c$ is the speed of light, and $\sigma_{0}=1.76\times10^{-44}\,{\rm cm^{2}}$; the energy moments

$$
\varepsilon_{N}(\pm\etae) = \frac{8\pi}{(hc)^{3}}T^{N+1}F_{N}(\pm\etae)\;,
$$

where $h$ is Planck's constant; and the Fermi-Dirac integrals

$$
F_{k}(\eta) = \int_{0}^{\infty}\dif x\frac{x^{k}}{e^{x-\eta}+1}\;.
$$

In the blocking factor, the electron neutrino and antineutrino degeneracy parameters are computed using

$$
\begin{align}
\eta_{\nue} &= \eta_{\nue}^{\rm ceq} \left[1 - \exp(-\tau_{\nue})\right] + \eta_{\nue}^{0}\exp(-\tau_{\nue})\; ,\\
\eta_{\anue} &= \eta_{\anue}^{\rm ceq} \left[1 - \exp(-\tau_{\anue})\right] + \eta_{\anue}^{0}\exp(-\tau_{\anue})\; ,
\end{align}
$$

where $\eta_{\nue}^{0} = 0 = \eta_{\anue}^{0}$, $\tau_{\nue}$ and $\tau_{\anue}$ are the optical depths of the electron neutrino and antineutrino, respectively, and

$$
\eta_{\nue}^{\rm ceq} =  - \eta_{\anue}^{\rm ceq} = \eta_{\rm e} + \eta_{\rm p} - \eta_{\rm n} - Q/T\; ,
$$

where $\eta_{i}=\mu_{i}/T$, with $i={\rm e,n,p}$, and $Q=1.2935$ MeV is the rest-mass-energy difference between a neutron and a proton. Finally, the number fractions $Y_{\rm np}$ and $Y_{\rm pn}$ are defined as

$$
\begin{align}
Y_{\rm pn} &=
\left\{
\begin{array}{c}
\frac{2\ye-1}{1-e^{\etahat}}\;, &\text{if }\ye<0.5\;,\\
1-\ye\;,&\text{otherwise}\;,
\end{array}
\right.\\
%%%%%%%%%%
Y_{\rm np} &=
\left\{
\begin{array}{c}
1-\ye\;, &\text{if }\ye<0.5\;,\\
e^{\etahat}Y_{\rm pn}\;,&\text{otherwise}\;.
\end{array}
\right.
\end{align}
$$

We declare $Y_{\rm np}$ and $Y_{\rm pn}$ as SymPy symbols below, computing them "by hand" in the C code before they are needed.

<a id='degeneracy_parameters'></a>

### Step 4.a.i: Compute all degeneracy parameters \[Back to [Top](#toc)\]
$$\label{degeneracy_parameters}$$

We start by computing the degeneracy parameters

$$
\eta_{i} = \mu_{i}/T\;,
$$

for $i=e,n,p$.

In [4]:
# Step 4: Free neutrino emission and cooling rates
# Step 4.a: Electron and positron capture (beta processes)
# Step 4.a.i: Degeneracy parameters
# Step 4.a.i.A: Declare SymPy symbols required to compute degeneracy parameters
zero      = sp.sympify(0)
mu_e, mu_n, mu_p, muhat = sp.symbols("mu_e mu_n mu_p muhat",real=True)
Y_np, Y_pn, T = sp.symbols("Y_np Y_pn T",real=True)
tau_0_nue , tau_1_nue  = sp.symbols("tau_nue[0]  tau_nue[1]" ,real=True)
tau_0_anue, tau_1_anue = sp.symbols("tau_anue[0] tau_anue[1]",real=True)
tau_0_nux , tau_1_nux  = sp.symbols("tau_nux[0]  tau_nux[1]" ,real=True)

eta_nue_0  = sp.symbols(thismodule+"_eta_nue_0",real=True)
eta_anue_0 = sp.symbols(thismodule+"_eta_anue_0",real=True)

# Step 4.a.i.B: Electron degeneracy parameter
eta_e = mu_e / T

# Step 4.a.i.C: Neutron degeneracy parameter
eta_n = mu_n / T

# Step 4.a.i.D: Proton degeneracy parameter
eta_p = mu_p / T

# Step 4.a.i.E: Neutron-proton difference degeneracy parameter
eta_hat = muhat / T

Now the neutrino degeneracy parameters:

$$
\begin{align}
\eta_{\nue} &= \eta_{\nue}^{\rm ceq} \left[1 - \exp(-\tau_{\nue})\right] + \eta_{\nue}^{0}\exp(-\tau_{\nue})\; ,\\
\eta_{\anue} &= \eta_{\anue}^{\rm ceq} \left[1 - \exp(-\tau_{\anue})\right] + \eta_{\anue}^{0}\exp(-\tau_{\anue})\; ,\\
\eta_{\nux} &= \eta_{\anux} = 0\;,
\end{align}
$$

where

$$
\eta_{\nue}^{\rm ceq} =  - \eta_{\anue}^{\rm ceq} = \eta_{\rm e} + \eta_{\rm p} - \eta_{\rm n} - Q/T\; .
$$

In [5]:
# Step 4.a.i.F: Electron neutrino degeneracy parameter (in equilibrium)
eta_nue_ceq = eta_e - eta_hat

# Step 4.a.i.G: Electron antineutrino degeneracy parameter (in equilibrium)
eta_anue_ceq = -eta_nue_ceq

# Step 4.a.i.H: Heavy lepton neutrinos/antineutrinos degeneracy parameter
eta_nux = zero

# Step 4.a.i.I: Electron neutrino degeneracy parameter
eta_nue = eta_nue_ceq#*(one-sp.exp(-tau_0_nue)) + eta_nue_0*sp.exp(-tau_0_nue)

# Step 4.a.i.J: Electron antineutrino degeneracy parameter
eta_anue = eta_anue_ceq#*(one-sp.exp(-tau_0_anue)) + eta_anue_0*sp.exp(-tau_0_anue)

<a id='energy_moments'></a>

### Step 4.a.ii: Compute energy-moments \[Back to [Top](#toc)\]
$$\label{energy_moments}$$

We now compute the needed energy moments, $\varepsilon_{4}(+\etae)$ and $\varepsilon_{4}(-\etae)$, using

$$
\varepsilon_{N}(\pm\etae) = \frac{8\pi}{(hc)^{3}}T^{N+1}F_{N}(\pm\etae)\;.
$$

In [6]:
# Step 4.a.ii: Compute the electron energy moments
# Step 4.a.ii.A: Declare Fermi-Dirac integral sympy function
Fermi_Dirac_integral = outC.nrpyFermiDiracintegrals

# Step 4.a.ii.B: Function to compute energy moment
M_PI = sp.Symbol("M_PI",real=True)
def energy_moment_N(N,eta_e):
    eps_N = sp.sympify(8)*M_PI/NRPy_hc3 * T**(N+1) * Fermi_Dirac_integral(N,eta_e)
    return eps_N

# Step 4.a.ii.C: Compute energy moments
eps_4_plus_etae  = energy_moment_N(4,+eta_e)
eps_4_minus_etae = energy_moment_N(4,-eta_e)

<a id='beta_blocking_factors'></a>

### Step 4.a.iii: Blocking factors \[Back to [Top](#toc)\]
$$\label{beta_blocking_factors}$$

Now we compute the blocking factors:

$$
\begin{align}
\langle1-f_{\nue}\rangle_{\rm ec} &= \left\{1 + \exp\left[\eta_{\nue}-\frac{F_{5}(\eta_{\rm e})}{F_{4}(\eta_{\rm e})}\right]\right\}^{-1}\; ,\\
\langle1-f_{\nue}\rangle_{\rm pc} &= \left\{1 + \exp\left[\eta_{\anue}-\frac{F_{5}(-\eta_{\rm e})}{F_{4}(-\eta_{\rm e})}\right]\right\}^{-1}\; .
\end{align}
$$

In [7]:
# Step 4.a.iii: Blocking factors
F_4_plus_etae      = Fermi_Dirac_integral(4,+eta_e)
F_5_plus_etae      = Fermi_Dirac_integral(5,+eta_e)
F_4_minus_etae     = Fermi_Dirac_integral(4,-eta_e)
F_5_minus_etae     = Fermi_Dirac_integral(5,-eta_e)
blocking_factor_ec = one/(one + sp.exp(eta_nue  - F_5_plus_etae/F_4_plus_etae))
blocking_factor_pc = one/(one + sp.exp(eta_anue - F_5_minus_etae/F_4_minus_etae))

<a id='beta_emission_rates'></a>

### Step 4.a.iv: Free neutrino emission rates \[Back to [Top](#toc)\]
$$\label{beta_emission_rates}$$

We now compute the free neutrino emission rates for electron capture by a proton and positron capture by a neutron:

$$
\begin{align}
  \rate{\RR}{\nue}{\rm ec} &= \frac{1+3\alpha^{2}}{8}\beta\,N_{A}\rhob Y_{\rm pn}\varepsilon_{4}(+\etae)\langle1-f_{\nue}\rangle_{\rm ec}\;,\\
  %%%%
  \rate{\RR}{\anue}{\rm pc} &= \frac{1+3\alpha^{2}}{8}\beta\,N_{A}\rhob Y_{\rm np}\varepsilon_{4}(-\etae)\langle1-f_{\anue}\rangle_{\rm pc}\;,
\end{align}
$$

In [8]:
# Step 4.a.iv: Free neutrino emission rates
rho_b       = sp.symbols("rho_b_cgs",real=True)
beta        = sp.symbols(thismodule+"_beta",real=True)
R_beta_nue  = sp.Rational(1,8)*(one + sp.sympify(3)*NRPy_alpha**2)*beta*NRPy_N_A*rho_b*Y_pn*eps_4_plus_etae*blocking_factor_ec
R_beta_anue = sp.Rational(1,8)*(one + sp.sympify(3)*NRPy_alpha**2)*beta*NRPy_N_A*rho_b*Y_np*eps_4_minus_etae*blocking_factor_pc

R_beta_nue  *= enable_beta_nue
R_beta_anue *= enable_beta_anue

<a id='pair_annihilation'></a>

## Step 4.b: Electron-positron pair annihilation \[Back to [Top](#toc)\]
$$\label{pair_annihilation}$$

We now consider the blocking factor for electron-positron pair annihilation, i.e.,

$$
\ae + \ee \to \nui + \anui\; ,
$$

which is given by Eq. (B9) in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R),

$$
\begin{align}
  \rate{\RR}{\nue,\anue}{\ee\ae} &= \frac{\bigl(C_{1}+C_{2}\bigr)_{\nue,\anue}}{36}\beta\,\varepsilon_{3}(\etae)\varepsilon_{3}(-\etae)\langle1-f_{\nue}\rangle_{\ee\ae}\langle1-f_{\anue}\rangle_{\ee\ae}\;,\\
  %%%%%%
  \rate{\RR}{\nux,\anux}{\ee\ae} &= \frac{\bigl(C_{1}+C_{2}\bigr)_{\nux,\anux}}{36}\beta\,\varepsilon_{3}(\etae)\varepsilon_{3}(-\etae)\bigl(\langle1-f_{\nux}\rangle_{\ee\ae}\bigr)^{2}\;,
\end{align}
$$

where

$$
\begin{align}
  \bigl(C_{1}+C_{2}\bigr)_{\nue,\anue}&=\bigl(C_V-C_{A}\bigr)^{2} + \bigl(C_{V}+C_{A}\bigr)^{2}\;,\\
  %%%%%%%
  \bigl(C_{1}+C_{2}\bigr)_{\nux,\anux}&=\bigl(C_V-C_{A}\bigr)^{2} + \bigl(C_{V}+C_{A}-2\bigr)^{2}\;,
\end{align}
$$

with $C_{A}=\frac{1}{2}$ and $C_{V} = \frac{1}{2} + 2\sin^{2}\theta_{\rm W}\approx0.96$. The blocking factors are computed using

$$
\langle1-f_{\nui}\rangle_{\ee\ae} = \left\{1 + \exp\left[\eta_{\nui} - \frac{1}{2}\frac{F_{4}(\etae)}{F_{3}(\etae)}-\frac{1}{2}\frac{F_{4}(-\etae)}{F_{3}(-\etae)}\right]\right\}^{-1}\;,
$$

with the blocking factor for antineutrinos obtained by replacing $\nui\to\anui$.

In [9]:
# Step 4.b: Electron positron pair annihilation
# Step 4.b.i: Fermi integrals
F_3_plus_etae  = Fermi_Dirac_integral(3,+eta_e)
F_3_minus_etae = Fermi_Dirac_integral(3,-eta_e)

# Step 4.b.ii: Blocking factors
common_term = sp.Rational(1,2)*(F_4_plus_etae/F_3_plus_etae + F_4_minus_etae/F_3_minus_etae)
blocking_factor_pair_nue  = one/(one + sp.exp(eta_nue  - common_term))
blocking_factor_pair_anue = one/(one + sp.exp(eta_anue - common_term))
blocking_factor_pair_nux  = one/(one + sp.exp(eta_nux  - common_term))

# Step 4.b.iii: Energy-moments
eps_3_plus_etae  = energy_moment_N(3,+eta_e)
eps_3_minus_etae = energy_moment_N(3,-eta_e)

# Step 4.b.iv: Constants
C1pC2_nue_anue = sp.symbols(thismodule+"_C1pC2_nue_anue",real=True)
C1pC2_nux_anux = sp.symbols(thismodule+"_C1pC2_nux_anux",real=True)

# Step 4.b.v: Free emission rates
common_term     = sp.Rational(1,36)*beta*eps_3_plus_etae*eps_3_minus_etae
R_pair_nue_anue = C1pC2_nue_anue*common_term*blocking_factor_pair_nue*blocking_factor_pair_anue
R_pair_nux_anux = C1pC2_nux_anux*common_term*blocking_factor_pair_nux**2

R_pair_nue_anue *= enable_pair_nue_anue
R_pair_nux_anux *= enable_pair_nux_anux

<a id='plasmon_decay'></a>

## Step 4.c: Transverse plasmon decay \[Back to [Top](#toc)\]
$$\label{plasmon_decay}$$

Next we compute the free neutrino emission rates for plasmon decay:

$$
\begin{align}
  \rate{\RR}{\nue,\anue}{\gamma} = \frac{\pi^{3}}{3\alpha_{\rm fs}}C_{V}^{2}\beta\,\frac{T^{8}}{(hc)^{6}}\gamma^{6}e^{-\gamma}(1+\gamma)\langle1-f_{\nue}\rangle_{\gamma}\langle1-f_{\anue}\rangle_{\gamma}\;,
  \label{eq:plasmon_emission_rate_nue_anue}\\
  %%%%%%%
  \rate{\RR}{\nux,\anux}{\gamma} = \frac{\pi^{3}}{3\alpha_{\rm fs}}\bigl(C_{V}-1\bigr)^{2}\beta\,\frac{T^{8}}{(hc)^{6}}\gamma^{6}e^{-\gamma}(1+\gamma)\bigl(\langle1-f_{\nux}\rangle_{\gamma}\bigr)^{2}\;,
  \label{eq:plasmon_emission_rate_nux_anux}
\end{align}
$$

where $\alpha_{\rm fs}=1/137.036$ is the fine-structure constant, $\gamma=\gamma_{0}\sqrt{\frac{1}{3}\bigl(\pi^{2}+3\etae^{2}\bigr)}$, with $\gamma_{0}=\hslash\Omega_{0}/\me c^{2} = 5.565\times10^{-2}$ related to the plasma frequency. The blocking factors are given by

$$
  \langle1-f_{\nui}\rangle_{\gamma} = \left\{1 + \exp\left[\eta_{\nui} - \left(1 + \frac{1}{2}\frac{\gamma^{2}}{1+\gamma}\right)\right]\right\}^{-1}\;,
$$

with the blocking factor for antineutrinos again obtained by replacing $\nui\to\anui$.

In [10]:
# Step 4.c: Transverse plasmon decay
# Step 4.c.i: Common terms
gamma = NRPy_gamma_0*sp.sqrt(sp.Rational(1,3)*(M_PI**2 + sp.sympify(3)*eta_e**2))
common_term = sp.Rational(1,3) * M_PI**3 * beta * T**8 * gamma**6 * sp.exp(-gamma) * (one+gamma) / ( NRPy_alpha_fs * NRPy_hc3**2 )

# Step 4.c.ii: Blocking factors
gamma_term = one + sp.Rational(1,2)*gamma**2/(one+gamma)
blocking_factor_plasmon_nue  = one/(one + sp.exp(eta_nue  - gamma_term))
blocking_factor_plasmon_anue = one/(one + sp.exp(eta_anue - gamma_term))
blocking_factor_plasmon_nux  = one/(one + sp.exp(eta_nux  - gamma_term))

# Step 4.c.iii: Free emission rates
C_V = sp.symbols(thismodule+"_C_V",real=True)
R_plasmon_nue_anue = common_term * C_V**2 * blocking_factor_plasmon_nue * blocking_factor_plasmon_anue
R_plasmon_nux_anux = common_term * (C_V-1)**2 * blocking_factor_plasmon_nux**2

R_plasmon_nue_anue *= enable_plasmon_nue_anue
R_plasmon_nux_anux *= enable_plasmon_nux_anux

<a id='bremsstrahlung'></a>

## Step 4.d: Nucleon-nucleon Bremsstrahlung \[Back to [Top](#toc)\]
$$\label{bremsstrahlung}$$

Finally, we compute the free neutrino emission rates for nucleon-nucleon Bremsstrahlung:

$$
\rate{\RR}{\nui,\anui}{\rm Bremss} = C_{1}\zeta\left(X_{\rm n}^{2} + X_{\rm p}^{2} + \frac{28}{3}X_{\rm n}X_{\rm p}\right)\rhob T^{4.5}\;,
$$

where $C_{1}=2.9988\times10^{7}$, $X_{\rm n}$ and $X_{\rm p}$ are the mass fractions of the neutron and the proton, respectively, and $\zeta=0.5$ is an adhoc parameter that controls the overall amplitude of the emission rate.

In [11]:
# Step 4.d: Nucleon-nucleon Bremsstrahlung
X_n, X_p = sp.symbols("X_n X_p",real=True)
R_Brems_nui_anui = NRPy_Brems_C1 * NRPy_Brems_zeta * ( X_n**2 + X_p**2 + sp.Rational(28,3)*X_n*X_p ) * rho_b * T**4.5
R_Brems_nui_anui *= enable_brems_nui_anui

<a id='free_cooling_rates'></a>

## Step 4.e: Free neutrino cooling rates \[Back to [Top](#toc)\]
$$\label{free_cooling_rates}$$

The corresponding neutrino cooling rates are computed using

$$
\begin{align}
  \rate{\QQ}{\nue}{\rm ec} &= \rate{\RR}{\nue}{\rm ec} \frac{\varepsilon_{5}(\etae)}{\varepsilon_{4}(\etae)}\;,\\
  %%%%%%
  \rate{\QQ}{\anue}{\rm ec} &= \rate{\RR}{\anue}{\rm pc} \frac{\varepsilon_{5}(-\etae)}{\varepsilon_{4}(-\etae)}\;,\\
  %%%%%%
  \rate{\QQ}{\nui,\anui}{\ee\ae} &= \rate{\RR}{\nui,\anui}{\ee\ae} \frac{1}{2}\frac{\varepsilon_{4}(\etae)\varepsilon_{3}(-\etae) + \varepsilon_{3}(\etae)\varepsilon_{4}(-\etae)}{\varepsilon_{3}(\etae)\varepsilon_{3}(-\etae)}\;,\\
  %%%%%%
  \rate{\QQ}{\nui,\anui}{\gamma} &= \rate{\RR}{\nui,\anui}{\gamma} \frac{1}{2}T\left(2+\frac{\gamma^{2}}{1+\gamma}\right)\;,\\
  %%%%%%
  \rate{\QQ}{\nui,\anui}{\rm Bremss} &= \rate{\RR}{\nui,\anui}{\rm Bremss}T\frac{C_{2}}{C_{1}}\;,
\end{align}
$$

where $C_{2} = 6.5428\times10^{7}$.

In [12]:
# Step 4.e: Free neutrino cooling rates
# Step 4.e.i: Energy-moments
eps_5_plus_etae  = energy_moment_N(5,+eta_e)
eps_5_minus_etae = energy_moment_N(5,-eta_e)

# Step 4.e.ii: Electron capture
Q_beta_nue = R_beta_nue * eps_5_plus_etae/eps_4_plus_etae

# Step 4.e.iii: Positron capture
Q_beta_anue = R_beta_anue * eps_5_minus_etae/eps_4_minus_etae

# Step 4.e.iv: Electron-positron pair annihilation
common_term = sp.Rational(1,2)*( eps_4_plus_etae*eps_3_minus_etae + eps_3_plus_etae*eps_4_minus_etae )/( eps_3_plus_etae*eps_3_minus_etae )
Q_pair_nue_anue = R_pair_nue_anue * common_term
Q_pair_nux_anux = R_pair_nux_anux * common_term

# Step 4.e.v: Transverse plasmon decay
common_term = sp.Rational(1,2)*T*(sp.sympify(2)+gamma**2/(one+gamma))
Q_plasmon_nue_anue = R_plasmon_nue_anue * common_term
Q_plasmon_nux_anux = R_plasmon_nux_anux * common_term

# Step 4.e.vi: Nucleon-nucleon Bremsstrahlung
Q_Brems_nui_anui = R_Brems_nui_anui * T * NRPy_Brems_C2 / NRPy_Brems_C1

<a id='total_rates'></a>

## Step 4.f: Total emission and cooling rates for free neutrinos \[Back to [Top](#toc)\]
$$\label{total_rates}$$

Finally, we compute the total emission and cooling rates for free neutrinos:

$$
\begin{align}
  \rate{\RR}{\nue}{\rm total} &= \rate{\RR}{\nue,\anue}{\ee\ae}
                              + \rate{\RR}{\nue,\anue}{\gamma}
                              + \rate{\RR}{\nue,\anue}{\rm Bremss}
                              + \rate{\RR}{\nue}{\rm ec}\;,\\
%%%%%%
  \rate{\RR}{\anue}{\rm total} &= \rate{\RR}{\nue,\anue}{\ee\ae}
                               + \rate{\RR}{\nue,\anue}{\gamma}
                               + \rate{\RR}{\nue,\anue}{\rm Bremss}
                               + \rate{\RR}{\anue}{\rm pc}\;,\\
%%%%%%
  \rate{\RR}{\nux}{\rm total} &= \rate{\RR}{\nux,\anux}{\ee\ae}
                              + \rate{\RR}{\nux,\anux}{\gamma}
                              + \rate{\RR}{\nux,\anux}{\rm Bremss}\;,\\
%%%%%%
  \rate{\QQ}{\nue}{\rm total} &= \rate{\QQ}{\nue,\anue}{\ee\ae}
                              + \rate{\QQ}{\nue,\anue}{\gamma}
                              + \rate{\QQ}{\nue,\anue}{\rm Bremss}
                              + \rate{\QQ}{\nue}{\rm ec}\;,\\
%%%%%%
  \rate{\QQ}{\anue}{\rm total} &= \rate{\QQ}{\nue,\anue}{\ee\ae}
                               + \rate{\QQ}{\nue,\anue}{\gamma}
                               + \rate{\QQ}{\nue,\anue}{\rm Bremss}
                               + \rate{\QQ}{\anue}{\rm pc}\;,\\
%%%%%%
  \rate{\QQ}{\nux}{\rm total} &= \rate{\QQ}{\nux,\anux}{\ee\ae}
                              + \rate{\QQ}{\nux,\anux}{\gamma}
                              + \rate{\QQ}{\nux,\anux}{\rm Bremss}\;,
\end{align}
$$

In [13]:
# Step 5: Total emission and cooling rates for free neutrinos

# Step 5.a: Electron neutrinos
R_free_total_nue  = R_pair_nue_anue + R_plasmon_nue_anue + R_Brems_nui_anui + R_beta_nue
Q_free_total_nue  = Q_pair_nue_anue + Q_plasmon_nue_anue + Q_Brems_nui_anui + Q_beta_nue

# Step 5.b: Electron antineutrinos
R_free_total_anue = R_pair_nue_anue + R_plasmon_nue_anue + R_Brems_nui_anui + R_beta_anue
Q_free_total_anue = Q_pair_nue_anue + Q_plasmon_nue_anue + Q_Brems_nui_anui + Q_beta_anue

# Step 5.c: Heavy lepton neutrinos or antineutrinos (single species)
R_free_total_nux  = R_pair_nux_anux + R_plasmon_nux_anux + R_Brems_nui_anui
Q_free_total_nux  = Q_pair_nux_anux + Q_plasmon_nux_anux + Q_Brems_nui_anui

<a id='grmhd_source_terms'></a>

# Step 5: GRMHD source terms \[Back to [Top](#toc)\]
$$\label{grmhd_source_terms}$$

We now implement the function that computes the GRMHD source terms $\RR$ and $\QQ$. These follow respectively from Eqs. (B24) and (B25) of [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R),

$$
\begin{align}
\RR &\equiv -\rate{\RR}{\nue}{\rm eff} + \rate{\RR}{\anue}{\rm eff}\;,\\
\QQ &\equiv -\left(\rate{\QQ}{\nue}{\rm eff} + \rate{\QQ}{\anue}{\rm eff} + 4\rate{\QQ}{\nux}{\rm eff}\right)\;,
\end{align}
$$

where \[cf. Eq. B22 of [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R)\]

$$
\rate{\RR}{\nui}{\rm eff} \equiv \frac{\rate{\RR}{\nui}{\rm total}}{1 + t^{\rm diff}_{\nui,0}\bigl(t^{\rm loss}_{\nui,0}\bigr)^{-1}}\;,
$$

and \[cf. Eq. B23 of [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R)\]

$$
\rate{\QQ}{\nui}{\rm eff} \equiv \frac{\rate{\QQ}{\nui}{\rm total}}{1 + t^{\rm diff}_{\nui,1}\bigl(t^{\rm loss}_{\nui,1}\bigr)^{-1}}\;.
$$

In the above equations, $t^{\rm diff}_{\nui,0}$ and $t^{\rm diff}_{\nui,1}$ are diffusion time scales associated with the emission and cooling rates, while $t^{\rm loss}_{\nui,0}$ and $t^{\rm loss}_{\nui,1}$ are time scales associated with the direct loss of neutrinos due to their free emission. We postpone the discussion of the computation of the diffusion time scales to Step 6 below. Here we compute the time scales associated with direct loss of neutrinos, which are given by Eqs. (B20) and (B21) of [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R)

$$
\begin{align}
\bigl(t^{\rm loss}_{\nui,0}\bigr)^{-1}\equiv\frac{\rate{\RR}{\nui}{\rm total}}{n_{\nui}}\;,\\
\bigl(t^{\rm loss}_{\nui,1}\bigr)^{-1}\equiv\frac{\rate{\QQ}{\nui}{\rm total}}{\epsilon_{\nui}}\;,
\end{align}
$$

where

$$
n_{\nui} \equiv \frac{4\pi}{(hc)^{3}}T^{3}F_{2}(\eta_{\nui})\;,
$$

and

$$
\epsilon_{\nui} \equiv \frac{4\pi}{(hc)^{3}}T^{4}F_{3}(\eta_{\nui})\;,
$$

are the neutrino number and energy densities. Note that for heavy lepton neutrinos and antineutrinos, $\nux$, $\rate{\RR}{\nux}{\rm total}$ and $\rate{\QQ}{\nux}{\rm total}$ are the rates for a **single neutrino species**, and thus we do not need the factors of $g_{\nui}$ in $n_{\nui}$ and $\epsilon_{\nui}$, but we do need the factor of $4$ that appears in the definition of the source term $\QQ$ above.

<a id='neutrino_number_and_energy_densities'></a>

## Step 5.a: Neutrino number and energy densities \[Back to [Top](#toc)\]
$$\label{neutrino_number_and_energy_densities}$$

Below we implement the neutrino number and energy densities, respectively given by

$$
\begin{align}
n_{\nui} &\equiv \frac{4\pi}{(hc)^{3}}T^{3}F_{2}(\eta_{\nui})\;,\\
\epsilon_{\nui} &\equiv \frac{4\pi}{(hc)^{3}}T^{4}F_{3}(\eta_{\nui})\;.
\end{align}
$$

We only need $n_{\nue}$ and $n_{\anue}$, but we need $\epsilon_{\nui}$ for all neutrino species.

In [14]:
# Step 5: GRMHD source terms
# Step 5.a: Neutrino number and energy densities
four = sp.sympify(4)
# Step 5.a.i: Electron neutrino number density
number_density_nue  = (four*M_PI/NRPy_hc3) * T**3 * Fermi_Dirac_integral(2,eta_nue)

# Step 5.a.ii: Electron antineutrino number density
number_density_anue = (four*M_PI/NRPy_hc3) * T**3 * Fermi_Dirac_integral(2,eta_anue)

# Step 5.a.iii: Electron neutrino energy density
energy_density_nue  = (four*M_PI/NRPy_hc3) * T**4 * Fermi_Dirac_integral(3,eta_nue)

# Step 5.a.iv: Electron antineutrino energy density
energy_density_anue = (four*M_PI/NRPy_hc3) * T**4 * Fermi_Dirac_integral(3,eta_anue)

# Step 5.a.v: Heavy lepton neutrinos energy density
energy_density_nux  = (four*M_PI/NRPy_hc3) * T**4 * Fermi_Dirac_integral(3,eta_nux)

<a id='direct_loss_time_scales'></a>

## Step 5.b: Direct neutrino loss time scales \[Back to [Top](#toc)\]
$$\label{direct_loss_time_scales}$$

We now implement the direct neutrino loss time scales:

$$
\begin{align}
\bigl(t^{\rm loss}_{\nui,0}\bigr)^{-1}\equiv\frac{\rate{\RR}{\nui}{\rm total}}{n_{\nui}}\;,\\
\bigl(t^{\rm loss}_{\nui,1}\bigr)^{-1}\equiv\frac{\rate{\QQ}{\nui}{\rm total}}{\epsilon_{\nui}}\;.
\end{align}
$$

We only need $\bigl(t^{\rm loss}_{\nue,0}\bigr)^{-1}$ and $\bigl(t^{\rm loss}_{\anue,0}\bigr)^{-1}$, but $\bigl(t^{\rm loss}_{\nui,1}\bigr)^{-1}$ must be computed for all neutrino species.

In [15]:
# Step 5.b: Direct neutrino loss time scales
# Step 5.b.i: Time scales associated with neutrino number rates
# Step 5.b.i.A: Electron neutrino
inv_t_loss_0_nue  = R_free_total_nue  / number_density_nue

# Step 5.b.i.B: Electron antineutrino
inv_t_loss_0_anue = R_free_total_anue / number_density_anue

# Step 5.b.ii: Time scales associated with neutrino cooling rates
# Step 5.b.ii.A: Electron neutrino
inv_t_loss_1_nue  = Q_free_total_nue  / energy_density_nue

# Step 5.b.ii.B: Electron antineutrino
inv_t_loss_1_anue = Q_free_total_anue / energy_density_anue

# Step 5.b.ii.C: Heavy lepton neutrinos
inv_t_loss_1_nux  = Q_free_total_nue  / energy_density_nux

<a id='scattering_opacities'></a>

## Step 5.c: Scattering opacities \[Back to [Top](#toc)\]
$$\label{scattering_opacities}$$

We compute the scattering opacities

$$
\kappa_{{\rm s},j}^{\nui,N} = C_{{\rm s},N}\sigma_{0}N_{A}\rhob Y_{NN}\left(\frac{T}{\me c^{2}}\right)^{2}\frac{F_{4+j}(\eta_{\nui})}{F_{2+j}(\eta_{\nui})}\;,
$$

where $j=0$ for neutrino number and $j=1$ for neutrino energy, and $C_{{\rm s},n} = (1+5\alpha^{2})/24$, $C_{{\rm s},p} = [4(C_{V}-1)^{2}+5\alpha^{2}]/24$, and

$$
Y_{NN} = \frac{Y_{N}}{1+\frac{2}{3}\max(\eta_{N},0)}\;,
$$

with $Y_{\rm n}=1-\ye$ and $Y_{\rm p}=\ye$.

In [16]:
# Step 5.c: Scattering opacities
# Step 5.c.i: Set basic constants
MAX = sp.Function("MAX")
Y_e = sp.Symbol("Y_e",real=True)
Y_p = Y_e
Y_n = one - Y_e
Y_pp = Y_p / ( one + sp.Rational(2,3)*MAX(eta_p,zero) )
Y_nn = Y_n / ( one + sp.Rational(2,3)*MAX(eta_n,zero) )
C_sn = sp.Rational(1,24)*(one + sp.sympify(5)*NRPy_alpha**2)
C_sp = sp.Rational(1,24)*(four*(C_V-1)**2 + sp.sympify(5)*NRPy_alpha**2)

# Step 5.c.ii: Common terms
common_term = NRPy_sigma_0*NRPy_N_A*rho_b*(T/NRPy_m_e_c2)**2

# Step 5.c.ii: Scattering opacities - neutron
kappa_s_n_0_nue  = C_sn*common_term*Y_nn*Fermi_Dirac_integral(4,eta_nue )/Fermi_Dirac_integral(2,eta_nue )
kappa_s_n_0_anue = C_sn*common_term*Y_nn*Fermi_Dirac_integral(4,eta_anue)/Fermi_Dirac_integral(2,eta_anue)
kappa_s_n_0_nux  = C_sn*common_term*Y_nn*Fermi_Dirac_integral(4,eta_nux )/Fermi_Dirac_integral(2,eta_nux )
kappa_s_n_1_nue  = C_sn*common_term*Y_nn*Fermi_Dirac_integral(5,eta_nue )/Fermi_Dirac_integral(3,eta_nue )
kappa_s_n_1_anue = C_sn*common_term*Y_nn*Fermi_Dirac_integral(5,eta_anue)/Fermi_Dirac_integral(3,eta_anue)
kappa_s_n_1_nux  = C_sn*common_term*Y_nn*Fermi_Dirac_integral(5,eta_nux )/Fermi_Dirac_integral(3,eta_nux )

# Step 5.c.ii: Scattering opacities - proton
kappa_s_p_0_nue  = C_sp*common_term*Y_pp*Fermi_Dirac_integral(4,eta_nue )/Fermi_Dirac_integral(2,eta_nue )
kappa_s_p_0_anue = C_sp*common_term*Y_pp*Fermi_Dirac_integral(4,eta_anue)/Fermi_Dirac_integral(2,eta_anue)
kappa_s_p_0_nux  = C_sp*common_term*Y_pp*Fermi_Dirac_integral(4,eta_nux )/Fermi_Dirac_integral(2,eta_nux )
kappa_s_p_1_nue  = C_sp*common_term*Y_pp*Fermi_Dirac_integral(5,eta_nue )/Fermi_Dirac_integral(3,eta_nue )
kappa_s_p_1_anue = C_sp*common_term*Y_pp*Fermi_Dirac_integral(5,eta_anue)/Fermi_Dirac_integral(3,eta_anue)
kappa_s_p_1_nux  = C_sp*common_term*Y_pp*Fermi_Dirac_integral(5,eta_nux )/Fermi_Dirac_integral(3,eta_nux )

<a id='absorption_opacities'></a>

## Step 5.d: Absorption opacities \[Back to [Top](#toc)\]
$$\label{absorption_opacities}$$

We compute the absorption opacities

$$
\begin{align}
\kappa_{a,j}^{\nue,n} &= \frac{1+3\alpha^{2}}{4}\sigma_{0}N_{A}\rhob Y_{\rm np}\left(\frac{T}{\me c^{2}}\right)^{2}\frac{F_{4+j}(\etanue)}{F_{2+j}(\etanue)}\langle1-f_{\ee}(\epsilon)\rangle\;,\\
\kappa_{a,j}^{\anue,p} &= \frac{1+3\alpha^{2}}{4}\sigma_{0}N_{A}\rhob Y_{\rm pn}\left(\frac{T}{\me c^{2}}\right)^{2}\frac{F_{4+j}(\etaanue)}{F_{2+j}(\etaanue)}\langle1-f_{\ae}(\epsilon)\rangle\;,
\end{align}
$$

where

$$
\begin{align}
\langle1-f_{\ee}(\epsilon)\rangle &= \left\{1+\exp\left[+\etae - \frac{F_{5}(\etanue)}{F_{4}(\etanue)}\right]\right\}^{-1}\;,\\
\langle1-f_{\ae}(\epsilon)\rangle &= \left\{1+\exp\left[-\etae - \frac{F_{5}(\etaanue)}{F_{4}(\etaanue)}\right]\right\}^{-1}\;.
\end{align}
$$

In [17]:
# Step 5.d: Absorption opacities
# Step 5.d.i: Blocking factors
bf_nue  = one/(one + sp.exp(+eta_e - Fermi_Dirac_integral(5,eta_nue )/Fermi_Dirac_integral(4,eta_nue )))
bf_anue = one/(one + sp.exp(-eta_e - Fermi_Dirac_integral(5,eta_anue)/Fermi_Dirac_integral(4,eta_anue)))

# Step 5.d.ii: Common term
common_term = sp.Rational(1,4)*(one+sp.sympify(3)*NRPy_alpha**2)*NRPy_sigma_0*NRPy_N_A*rho_b*(T/NRPy_m_e_c2)**2

# Step 5.d.iii: Absorption opacities
kappa_a_0_nue  = common_term*Y_np*Fermi_Dirac_integral(4,eta_nue )/Fermi_Dirac_integral(2,eta_nue )*bf_nue
kappa_a_1_nue  = common_term*Y_np*Fermi_Dirac_integral(5,eta_nue )/Fermi_Dirac_integral(3,eta_nue )*bf_nue
kappa_a_0_anue = common_term*Y_pn*Fermi_Dirac_integral(4,eta_anue)/Fermi_Dirac_integral(2,eta_anue)*bf_anue
kappa_a_1_anue = common_term*Y_pn*Fermi_Dirac_integral(5,eta_anue)/Fermi_Dirac_integral(3,eta_anue)*bf_anue

<a id='total_transport_opacities'></a>

## Step 5.e: Total transport opacities \[Back to [Top](#toc)\]
$$\label{total_transport_opacities}$$

Finally, we compute the total transport opacities:

$$
\begin{align}
\kappa_{{\rm t},j}^{\nue} &= \kappa_{{\rm s},j}^{\nue,n} + \kappa_{{\rm s},j}^{\nue,p} + \kappa_{{\rm a},j}^{\nue,n}\;,\\
\kappa_{{\rm t},j}^{\anue} &= \kappa_{{\rm s},j}^{\anue,n} + \kappa_{{\rm s},j}^{\anue,p} + \kappa_{{\rm a},j}^{\anue,p}\;,\\
\kappa_{{\rm t},j}^{\nux} &= \kappa_{{\rm s},j}^{\nux,n} + \kappa_{{\rm s},j}^{\nux,p}\;.
\end{align}
$$

Note also that opacities have units of inverse length, so we multiply them by

$$
\frac{1}{\mathtt{NRPy}\_\mathtt{Lcgstogeom}}=\mathtt{NRPy}\_\mathtt{Lgeomtocgs}\;,
$$

to obtain opacities in geometrized ($G=c=1$) units.

In [18]:
# Step 5.e: Total transport opacities
kappa_t_0_nue  = NRPy_Lgeomtocgs*(kappa_s_n_0_nue  + kappa_s_p_0_nue  + kappa_a_0_nue)
kappa_t_1_nue  = NRPy_Lgeomtocgs*(kappa_s_n_1_nue  + kappa_s_p_1_nue  + kappa_a_1_nue)
kappa_t_0_anue = NRPy_Lgeomtocgs*(kappa_s_n_0_anue + kappa_s_p_0_anue + kappa_a_0_anue)
kappa_t_1_anue = NRPy_Lgeomtocgs*(kappa_s_n_1_anue + kappa_s_p_1_anue + kappa_a_1_anue)
kappa_t_0_nux  = NRPy_Lgeomtocgs*(kappa_s_n_0_nux  + kappa_s_p_0_nux)
kappa_t_1_nux  = NRPy_Lgeomtocgs*(kappa_s_n_1_nux  + kappa_s_p_1_nux)

<a id='diffusion_timescales'></a>

## Step 5.f: Diffusion timescales \[Back to [Top](#toc)\]
$$\label{diffusion_timescales}$$

We now compute the diffusion time scales $t_{\rm diff}^{\nui}$ accordding to Eqs. (A31) and (A33) of [Rosswog and Liebendörfer (2003)](https://arxiv.org/abs/astro-ph/0302301) \[The extra factor of two is suggested in [O'Connor & Ott (2010)](https://arxiv.org/abs/0912.2393)\]:

$$
t_{{\rm diff},j}^{\nui} = \frac{6\left(\tau^{\nui}_{j}\right)^{2}}{c\kappa_{{\rm t},j}^{\nui}}\;.
$$

Note that in our implementation below we set $c=1$ since the opacities $\kappa_{{\rm t},j}^{\nui}$ have already been converted to geometrized units in the previous substep.

In [19]:
# Step 5.f: Diffusion timescales
six           = sp.sympify(6)
t_diff_0_nue  = six * tau_0_nue**2  / kappa_t_0_nue
t_diff_0_anue = six * tau_0_anue**2 / kappa_t_0_anue
t_diff_0_nux  = six * tau_0_nux**2  / kappa_t_0_nux
t_diff_1_nue  = six * tau_1_nue**2  / kappa_t_1_nue
t_diff_1_anue = six * tau_1_anue**2 / kappa_t_1_anue
t_diff_1_nux  = six * tau_1_nux**2  / kappa_t_1_nux

<a id='effective_emission_rates'></a>

## Step 5.g: Effective emission rates \[Back to [Top](#toc)\]
$$\label{effective_emission_rates}$$

Now we compute the effective emission rates:

$$
\begin{align}
\rate{\RR}{\nui}{\rm eff} &\equiv \frac{\rate{\RR}{\nui}{\rm total}}{1 + t^{\rm diff}_{\nui,0}\bigl(t^{\rm loss}_{\nui,0}\bigr)^{-1}}\;,\\
\rate{\QQ}{\nui}{\rm eff} &\equiv \frac{\rate{\QQ}{\nui}{\rm total}}{1 + t^{\rm diff}_{\nui,1}\bigl(t^{\rm loss}_{\nui,1}\bigr)^{-1}}\;.
\end{align}
$$

We only need $\rate{\RR}{\nue}{\rm eff}$ and $\rate{\RR}{\anue}{\rm eff}$, but we need $\rate{\QQ}{\nui}{\rm eff}$ for all neutrino species.

In [20]:
# Step 5.c: Effective emission rates
# Step 5.c.i: Effective number rates
# Step 5.c.i.A: Electron neutrino
R_eff_nue  = R_free_total_nue  / ( one + t_diff_0_nue *inv_t_loss_0_nue )

# Step 5.c.i.B: Electron antineutrino
R_eff_anue = R_free_total_anue / ( one + t_diff_0_anue*inv_t_loss_0_anue )

# Step 5.c.ii: Effective cooling rates
# Step 5.c.ii.A: Electron neutrino
Q_eff_nue  = Q_free_total_nue  / ( one + t_diff_1_nue *inv_t_loss_1_nue )

# Step 5.c.ii.B: Electron antineutrino
Q_eff_anue = Q_free_total_anue / ( one + t_diff_1_anue*inv_t_loss_1_anue )

# Step 5.c.ii.C: Heavy lepton neutrinos
Q_eff_nux  = Q_free_total_nux  / ( one + t_diff_1_nux *inv_t_loss_1_nux )

<a id='grmhd_source_terms_impl'></a>

## Step 5.h: Computing $\RR$ and $\QQ$ \[Back to [Top](#toc)\]
$$\label{grmhd_source_terms_impl}$$

Finally, we implement the GRMHD source terms:

$$
\begin{align}
\RR &\equiv -\rate{\RR}{\nue}{\rm eff} + \rate{\RR}{\anue}{\rm eff}\;,\\
\QQ &\equiv -\left(\rate{\QQ}{\nue}{\rm eff} + \rate{\QQ}{\anue}{\rm eff} + 4\rate{\QQ}{\nux}{\rm eff}\right)\;.
\end{align}
$$

Additionally, note that we write

$$
\begin{align}
\nabla_{\mu}\bigl(n_{\rm e} u^{\mu}\bigr) &= \RR\; ,\\
\nabla_{\mu}T^{\mu\nu} &= \QQ u^{\nu}\; ,
\end{align}
$$

so that

$$
\nabla_{\mu}\bigl(\ye \rhob u^{\mu}\bigr) = m_{\rm b}\RR\; .
$$

We absorb the factor of $m_{\rm b}=m_{\rm u}$, the atomic mass unit, in $\RR$ for simplicity. Because $\QQ\propto\RR$, we must also multiply $\QQ$ by the same factor. At this point we also convert the source terms from CGS to code (geometric) units.

Note: the units of $\RR$ are ${\rm g\,cm^{-3}\,s^{-1}}$, but $\ye$ is dimensionless. Therefore we must multiply $\RR$ by the factor

$$
\mathtt{Dcgstogeom} = \left(\frac{G_{\rm N}M_{\odot}}{c^{2}}\right)^{-1}\;,
$$

which has units of inverse density. Similarly, $\QQ$ has units of ${\rm MeV\,cm^{-3}\,s^{-1}}$, but $\epsilon$ has units of ${\rm erg\,g^{-1}=cm^{2}\,s^{-2}}$. The conversion from these units to code units involves three steps:

1. Multiply $\QQ$ by $\mathtt{Dcgstogeom}$ to obtain ${\rm MeV\,g^{-1}\,s^{-1}}$;
2. Multiply the result of step 1 by $\mathtt{MeV}\_\mathtt{to}\_\mathtt{erg}$ to obtain $\QQ$ in ${\rm erg\,g^{-1}\,s^{-1}=(cm^{2}\,s^{-2})s^{-1}}$;
3. Multiply the result of step 2 by $c^{-2}$ to obtain $\QQ$ in geometrized units.

In [21]:
# Step 5.d: GRMHD source terms
# Step 5.d.i: R
R_source = (R_eff_anue - R_eff_nue) * NRPy_amu * NRPy_Rcgstogeom

# Step 5.d.ii: Q
Q_source = -( Q_eff_nue + Q_eff_anue + four*Q_eff_nux ) * NRPy_Qcgstogeom

<a id='grmhd_source_terms_c_function_generation'></a>

## Step 5.i: C function generation \[Back to [Top](#toc)\]
$$\label{grmhd_source_terms_c_function_generation}$$

Now we generate the C code for the above expressions.

<a id='low_level_functions'></a>

### Step 5.i.i: Low level functions \[Back to [Top](#toc)\]
$$\label{low_level_functions}$$

In [22]:
def Cfunc_NRPyLeakage_compute_GRMHD_source_terms_and_opacities_all_different_constants():
    desc = """(c) Leo Werneck
Compute GRMHD source terms following Ruffert et al. (1996)
https://adsabs.harvard.edu/pdf/1996A%26A...311..532R"""
    if generate_ET_thorn:
        includes = ["cctk.h","NRPyLeakage.h","WVU_EOS_Tabulated_headers.h"]
    else:
        includes = ["Basic_defines.h","NRPyEOS.h","NRPyLeakage.h"]
    prefunc = ""
    c_type = "void"
    if generate_ET_thorn:
        params = """const REAL rho_b,
                    const REAL Y_e,
                    const REAL T,
                    const REAL *restrict tau_nue,
                    const REAL *restrict tau_anue,
                    const REAL *restrict tau_nux,
                    REAL *restrict R_source,
                    REAL *restrict Q_source,
                    REAL *restrict kappa_nue,
                    REAL *restrict kappa_anue,
                    REAL *restrict kappa_nux"""
    else:
        params = """const NRPyEOS_params *restrict eos_params,
                    const REAL rho_b,
                    const REAL Y_e,
                    const REAL T,
                    const REAL *restrict tau_nue,
                    const REAL *restrict tau_anue,
                    const REAL *restrict tau_nux,
                    REAL *restrict R_source,
                    REAL *restrict Q_source,
                    REAL *restrict kappa_nue,
                    REAL *restrict kappa_anue,
                    REAL *restrict kappa_nux"""
    body = """
  // Step 1: Get chemical potentials and mass
  //         fractions using the EOS
  REAL mu_e, mu_p, mu_n, muhat, X_p, X_n;"""
    if generate_ET_thorn:
        body += """
  WVU_EOS_mue_mup_mun_muhat_Xn_and_Xp_from_rho_Ye_T(rho_b, Y_e, T, &mu_e, &mu_p, &mu_n, &muhat, &X_p, &X_n);\n"""
    else:
        body += """
  NRPyEOS_mue_mup_mun_muhat_Xn_and_Xp_from_rho_Ye_T(eos_params, rho_b, Y_e, T,
                                                    &mu_e, &mu_p, &mu_n, &muhat, &X_p, &X_n);\n"""
    body += """
  // Step 2: Compute rho_b in cgs units
  const REAL rho_b_cgs = rho_b * NRPyLeakage_units_geom_to_cgs_D;

/* Leo says: this is the way ZelmaniLeak computes Y_pn and Y_np
  // Step 3: Compute Y_{pn} and Y_{np}
  // Step 3.a: Compute Y_{pn} (See discussion below Eq. A8 in https://arxiv.org/pdf/1306.4953.pdf)
  const REAL Y_p = Y_e;
  const REAL Y_n = 1-Y_e;
  REAL Y_pn;
  if( rho_b_cgs > 2e12 ) {
    // Step 3.a.i: Use Eqs. (A13) and (A14) in https://adsabs.harvard.edu/pdf/1996A%26A...311..532R
    Y_pn = (2*Y_e - 1)/(1-exp(muhat/T));
  }
  else {
    Y_pn = Y_n;
  }
  // Step 3.a.ii: Make sure Y_{pn} is nonzero
  Y_pn = MAX(Y_pn,0.0);
  
  // Step 3.b: Compute Y_{np} (Eq. A13 in https://adsabs.harvard.edu/pdf/1996A%26A...311..532R)
  const REAL Y_np = exp(muhat/T) * Y_pn;
*/

  // Step 3: Compute Y_{pn} and Y_{np}
  const REAL Y_p = Y_e;
  const REAL Y_n = 1-Y_e;
  const REAL exp_metahat = exp(-muhat/T);
  // Step 3.a: Compute Y_{np}
  REAL Y_np = (Y_e < 0.5) ? (2.0*Y_e-1.0)/(exp_metahat-1.0) : Y_n;

  // Step 3.b: Compute Y_{pn}
  REAL Y_pn = (Y_e > 0.5) ? exp_metahat*(2.0*Y_e-1.0)/(exp_metahat-1.0) : Y_p;

  // Step 3.c: Make sure both Y_np and Y_pn are non-negative
  if( Y_np < 0.0 ) Y_np = Y_n;
  if( Y_pn < 0.0 ) Y_pn = Y_p;

  // Step 4: Compute the source terms
  //         Note: The code below is generated by NRPy+
"""
    if debug_mode:
        body += """
  REAL R_beta_nue,R_beta_anue,R_pair_nue_anue,R_pair_nux_anux,R_plasmon_nue_anue,R_plasmon_nux_anux,R_Brems_nui_anui;
  REAL Q_beta_nue,Q_beta_anue,Q_pair_nue_anue,Q_pair_nux_anux,Q_plasmon_nue_anue,Q_plasmon_nux_anux,Q_Brems_nui_anui;
  REAL R_eff_nue,R_eff_anue,Q_eff_nue,Q_eff_anue,Q_eff_nux;
  REAL kappa_s_n_0_nue,kappa_s_p_0_nue,kappa_a_0_nue,kappa_s_n_1_nue,kappa_s_p_1_nue,kappa_a_1_nue;
  REAL kappa_s_n_0_anue,kappa_s_p_0_anue,kappa_a_0_anue,kappa_s_n_1_anue,kappa_s_p_1_anue,kappa_a_1_anue;
  REAL kappa_s_n_0_nux,kappa_s_p_0_nux,kappa_s_n_1_nux,kappa_s_p_1_nux;
"""
        cexpr = [R_beta_nue,R_beta_anue,R_pair_nue_anue,R_pair_nux_anux,R_plasmon_nue_anue,R_plasmon_nux_anux,R_Brems_nui_anui,
                 Q_beta_nue,Q_beta_anue,Q_pair_nue_anue,Q_pair_nux_anux,Q_plasmon_nue_anue,Q_plasmon_nux_anux,Q_Brems_nui_anui,
                 R_eff_nue,R_eff_anue,Q_eff_nue,Q_eff_anue,Q_eff_nux,
                 kappa_s_n_0_nue,kappa_s_p_0_nue,kappa_a_0_nue,
                 kappa_s_n_1_nue,kappa_s_p_1_nue,kappa_a_1_nue,
                 kappa_s_n_0_anue,kappa_s_p_0_anue,kappa_a_0_anue,
                 kappa_s_n_1_anue,kappa_s_p_1_anue,kappa_a_1_anue,
                 kappa_s_n_0_nux,kappa_s_p_0_nux,
                 kappa_s_n_1_nux,kappa_s_p_1_nux,
                 R_source,Q_source,kappa_t_0_nue, kappa_t_1_nue,kappa_t_0_anue,kappa_t_1_anue ,kappa_t_0_nux,kappa_t_1_nux]
        cvars = ["R_beta_nue","R_beta_anue","R_pair_nue_anue","R_pair_nux_anux","R_plasmon_nue_anue","R_plasmon_nux_anux","R_Brems_nui_anui",
                 "Q_beta_nue","Q_beta_anue","Q_pair_nue_anue","Q_pair_nux_anux","Q_plasmon_nue_anue","Q_plasmon_nux_anux","Q_Brems_nui_anui",
                 "R_eff_nue","R_eff_anue","Q_eff_nue","Q_eff_anue","Q_eff_nux",
                 "kappa_s_n_0_nue","kappa_s_p_0_nue","kappa_a_0_nue",
                 "kappa_s_n_1_nue","kappa_s_p_1_nue","kappa_a_1_nue",
                 "kappa_s_n_0_anue","kappa_s_p_0_anue","kappa_a_0_anue",
                 "kappa_s_n_1_anue","kappa_s_p_1_anue","kappa_a_1_anue",
                 "kappa_s_n_0_nux","kappa_s_p_0_nux",
                 "kappa_s_n_1_nux","kappa_s_p_1_nux",
                 "*R_source","*Q_source","kappa_nue[0]","kappa_nue[1]","kappa_anue[0]","kappa_anue[1]","kappa_nux[0]","kappa_nux[1]"]
    else:
        cexpr = [  R_source ,  Q_source , kappa_t_0_nue , kappa_t_1_nue , kappa_t_0_anue , kappa_t_1_anue , kappa_t_0_nux , kappa_t_1_nux ]
        cvars = ["*R_source","*Q_source", "kappa_nue[0]", "kappa_nue[1]", "kappa_anue[0]", "kappa_anue[1]", "kappa_nux[0]", "kappa_nux[1]"]
    outCparams = "outCverbose=False,includebraces=False,preindent=1"
    body += outC.outputC(cexpr,cvars,"returnstring",params=outCparams).replace("double","REAL")
    if debug_mode:
        max_length = len(cvars[0])
        for i in range(1,len(cvars)):
            if len(cvars[i]) > max_length:
                max_length = len(cvars[i])
        for i in range(len(cvars)):
            body += "  fprintf(stderr,\"(NRPyLeakage) "+cvars[i]
            for j in range(max_length-len(cvars[i])):
                body += " "
            body += " = %.15e\\n\","+cvars[i]+");\n"
        body += "  fprintf(stderr,\"(NRPyLeakage) ************************************************\\n\");\n"
    loopopts = ""
    
    global function_prototypes
    global make_code_defn_list
    
    # First output the function with NRPy parameters
    name   = "NRPyLeakage_compute_GRMHD_source_terms_and_opacities_nrpy_constants"
    if generate_ET_thorn:
        name += "_impl"
    params = adjust_params_indent(c_type,name,params)
    outC.outCfunction(os.path.join(Ccodesdir,name+".c"),
                      includes=includes,desc=desc,prefunc=prefunc,c_type=c_type,name=name,
                      params=params,body=body,loopopts=loopopts,enableCparameters=False)
    function_prototypes.append(c_type+" "+name+"("+params+");\n")
    function_prototypes = outC.superfast_uniq(function_prototypes)
    if generate_ET_thorn:
        make_code_defn_list.append(name+".c")

    # Then the function with HARM parameters
    name      = "NRPyLeakage_compute_GRMHD_source_terms_and_opacities_harm_constants"
    if generate_ET_thorn:
        name += "_impl"
    params    = adjust_params_indent(c_type,name,params)
    body_harm = body.replace("c_light","harm_c_light")
    body_harm = body_harm.replace("N_A","harm_N_A")
    body_harm = body_harm.replace("alpha_fs","harm_alpha_fs")
    body_harm = body_harm.replace("amu ","harm_amu ")
    body_harm = body_harm.replace("hc3","harm_hc3")
    body_harm = body_harm.replace("m_e_c2","harm_m_e_c2")
    body_harm = body_harm.replace("units_geom_","harm_units_geom_")
    body_harm = body_harm.replace("units_cgs_","harm_units_cgs_")

    outC.outCfunction(os.path.join(Ccodesdir,name+".c"),
                      includes=includes,desc=desc,prefunc=prefunc,c_type=c_type,name=name,
                      params=params,body=body_harm,loopopts=loopopts,enableCparameters=False)
    function_prototypes.append(c_type+" "+name+"("+params+");\n")
    function_prototypes = outC.superfast_uniq(function_prototypes)
    if generate_ET_thorn:
        make_code_defn_list.append(name+".c")

<a id='driver_function'></a>

### Step 5.i.ii: Driver function \[Back to [Top](#toc)\]
$$\label{driver_function}$$

In [23]:
def Cfunc_NRPyLeakage_compute_GRMHD_source_terms_and_opacities():
    desc = """(c) Leo Werneck
Compute GRMHD source terms following Ruffert et al. (1996)
https://adsabs.harvard.edu/pdf/1996A%26A...311..532R"""
    if generate_ET_thorn:
        includes = ["NRPyLeakage.h"]
    else:
        includes = ["Basic_defines.h","NRPyEOS.h","NRPyLeakage.h"]
    prefunc = ""
    c_type = "void"
    name   = "NRPyLeakage_compute_GRMHD_source_terms_and_opacities"
    if generate_ET_thorn:
        name += "_impl"
    if generate_ET_thorn:
        params = """const int which_constants_to_use,
                    const REAL rho_b,
                    const REAL Y_e,
                    const REAL T,
                    const REAL *restrict tau_nue,
                    const REAL *restrict tau_anue,
                    const REAL *restrict tau_nux,
                    REAL *restrict R_source,
                    REAL *restrict Q_source,
                    REAL *restrict kappa_nue,
                    REAL *restrict kappa_anue,
                    REAL *restrict kappa_nux"""
    else:
        params = """const int which_constants_to_use,
            const NRPyEOS_params *restrict eos_params,
            const REAL rho_b,
            const REAL Y_e,
            const REAL T,
            const REAL *restrict tau_nue,
            const REAL *restrict tau_anue,
            const REAL *restrict tau_nux,
            REAL *restrict R_source,
            REAL *restrict Q_source,
            REAL *restrict kappa_nue,
            REAL *restrict kappa_anue,
            REAL *restrict kappa_nux"""
    params = adjust_params_indent(c_type,name,params)
    body = r"""
  switch (which_constants_to_use) {"""
    if generate_ET_thorn:
        body += r"""
    case USE_NRPY_CONSTANTS:
      NRPyLeakage_compute_GRMHD_source_terms_and_opacities_nrpy_constants_impl(rho_b,Y_e,T,
                                                                               tau_nue,tau_anue,tau_nux,
                                                                               R_source,Q_source,
                                                                               kappa_nue,kappa_anue,kappa_nux);
      break;
    case USE_HARM_CONSTANTS:
      NRPyLeakage_compute_GRMHD_source_terms_and_opacities_harm_constants_impl(rho_b,Y_e,T,
                                                                               tau_nue,tau_anue,tau_nux,
                                                                               R_source,Q_source,
                                                                               kappa_nue,kappa_anue,kappa_nux);
      break;"""
    else:
        body += r"""
    case USE_NRPY_CONSTANTS:
      NRPyLeakage_compute_GRMHD_source_terms_and_opacities_nrpy_constants(eos_params,rho_b,Y_e,T,
                                                                          tau_nue,tau_anue,tau_nux,
                                                                          R_source,Q_source,
                                                                          kappa_nue,kappa_anue,kappa_nux);
      break;
    case USE_HARM_CONSTANTS:
      NRPyLeakage_compute_GRMHD_source_terms_and_opacities_harm_constants(eos_params,rho_b,Y_e,T,
                                                                          tau_nue,tau_anue,tau_nux,
                                                                          R_source,Q_source,
                                                                          kappa_nue,kappa_anue,kappa_nux);
      break;"""
    body += r"""
    default:
      fprintf(stderr,"(NRPyLeakage) ERROR: Unknown constant type (%d) in NRPyLeakage_compute_GRMHD_source_terms().\n",which_constants_to_use);
      fprintf(stderr,"(NRPyLeakage) Options are: USE_NRPY_CONSTANTS (%d) and USE_HARM_CONSTANTS (%d)\n",USE_NRPY_CONSTANTS,USE_HARM_CONSTANTS);
      fprintf(stderr,"(NRPyLeakage) Aborting!\n");
      exit(1);
  }
"""
    loopopts = ""
    outC.outCfunction(os.path.join(Ccodesdir,name+".c"),
                      includes=includes,desc=desc,prefunc=prefunc,c_type=c_type,name=name,
                      params=params,body=body,loopopts=loopopts,enableCparameters=False)

    global function_prototypes
    function_prototypes.append(c_type+" "+name+"("+params+");\n")
    function_prototypes = outC.superfast_uniq(function_prototypes)
    if generate_ET_thorn:
        global make_code_defn_list
        make_code_defn_list.append(name+".c")

<a id='optical_depth'></a>

# Step 6: Optical depth \[Back to [Top](#toc)\]
$$\label{optical_depth}$$

This function computes the optical depth by adopting a "path of least resistance" approach following [Nielsen *et al.* (2014)](https://arxiv.org/abs/1403.3680). This is also the approach adopted by `HARM`, as described in [Murguia-Berthier *et al.* (2021)](https://arxiv.org/abs/2106.05356).

In [24]:
def Cfunc_NRPyLeakage_compute_optical_depths():
    desc = """(c) Leo Werneck
Compute GRMHD source terms following Ruffert et al. (1996)
https://adsabs.harvard.edu/pdf/1996A%26A...311..532R"""
    if generate_ET_thorn:
        includes = ["NRPyLeakage.h"]
    else:
        includes = ["Basic_defines.h"]
    prefunc = """
#ifndef IDX3D
#define IDX3D(i0,i1,i2) ( (i0) + (N0)*( (i1) + (N1)*(i2) ) )
#endif
"""
    c_type = "void"
    name   = "NRPyLeakage_compute_optical_depths"
    if generate_ET_thorn:
        name += "_impl"
    params = """const int N0 , const int N1 , const int N2,
                const int Ng0, const int Ng1, const int Ng2,
                const int dxx0, const int dxx1, const int dxx2,
                const REAL *restrict gammaDD00,
                const REAL *restrict gammaDD11,
                const REAL *restrict gammaDD22,
                const REAL *restrict kappa_0_nue,
                const REAL *restrict kappa_1_nue,
                const REAL *restrict kappa_0_anue,
                const REAL *restrict kappa_1_anue,
                const REAL *restrict kappa_0_nux,
                const REAL *restrict kappa_1_nux,
                REAL *restrict tau_0_nue,
                REAL *restrict tau_1_nue,
                REAL *restrict tau_0_anue,
                REAL *restrict tau_1_anue,
                REAL *restrict tau_0_nux,
                REAL *restrict tau_1_nux"""
    params = adjust_params_indent(c_type,name,params)
    body = """
  // Step 0: Loop over the grid computing the optical depth
#pragma omp parallel for
  for(int i2=Ng2;i2<N2-Ng2;i2++) {
    for(int i1=Ng1;i1<N1-Ng1;i1++) {
      for(int i0=Ng0;i0<N0-Ng0;i0++) {\n
"""+tau_helpers.write_optical_depth_Cfunc_body(indent="        ")+"""
      } // for(int i0=Ng0;i0<N0-Ng0;i0++)
    } // for(int i1=Ng1;i1<N1-Ng1;i1++)
  } // for(int i2=Ng2;i2<N2-Ng2;i2++)
"""
    loopopts = ""
    outC.outCfunction(os.path.join(Ccodesdir,name+".c"),
                      includes=includes,desc=desc,prefunc=prefunc,c_type=c_type,name=name,
                      params=params,body=body,loopopts=loopopts,enableCparameters=False)

    global function_prototypes
    function_prototypes.append(c_type+" "+name+"("+params+");\n")
    function_prototypes = outC.superfast_uniq(function_prototypes)
    if generate_ET_thorn:
        global make_code_defn_list
        make_code_defn_list.append(name+".c")

<a id='writing_leakage_hh'></a>

# Step 7: Writing the C header file `Leakage.h` \[Back to [Top](#toc)\]
$$\label{writing_leakage_hh}$$

We now write the `Leakage.h` header file, which contains parameters and function prototypes for out Leakage scheme.

In [25]:
# Step 4.h: Creating the C header file Leakage.h
def Cheader_NRPyLeakage_h():

    # Step 4.h.i: First add all parameters which we set by hand
    NRPyLeakage_h_string = """#ifndef NRPYLEAKAGE_H_
#define NRPYLEAKAGE_H_

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define REAL double

#ifdef MIN
#undef MIN
#endif

#ifdef MAX
#undef MAX
#endif

#define MAX(a,b) ( (a) > (b) ? (a) : (b) )
#define MIN(a,b) ( (a) < (b) ? (a) : (b) )

// \"Primary\" parameters
"""
    for param in par.glb_Cparams_list:
        if param.module == thismodule:
            ctype = param.type
            cname = param.parname
            value = param.defaultval
            NRPyLeakage_h_string += ctype+" "+cname+" ("+value+")\n"

    # Step 4.h.ii: Now add derived parameters
    C_V_impl       = NRPy_C_A + sp.sympify(2)*NRPy_sinthw2
    beta           = NRPy_sigma_0 * NRPy_c_light / NRPy_m_e_c2**2
    harm_beta      = NRPy_sigma_0 * harm_c_light / harm_m_e_c2**2
    C1pC2_nue_anue = (C_V-NRPy_C_A)**2 + (C_V+NRPy_C_A)**2
    C1pC2_nux_anux = (C_V-NRPy_C_A)**2 + (C_V+NRPy_C_A-2)**2
    param_names  = ["C_V","beta","harm_beta","C1pC2_nue_anue","C1pC2_nux_anux"]
    param_values = [ C_V_impl , beta , harm_beta, C1pC2_nue_anue , C1pC2_nux_anux ]
    NRPyLeakage_h_string += "// \"Derived\" parameters\n"
    for i in range(len(param_names)):
        value = param_values[i]
        cname = "#define NRPyLeakage_"+param_names[i]
        Ccode = outC.outputC(value,cname,"returnstring",params="outCverbose=false,includebraces=false").replace("= ", "(").replace(";",")")
        NRPyLeakage_h_string += Ccode
    NRPyLeakage_h_string += "\n\n// Function prototypes\n"

    # Step 4.h.iii: Finally, add the function prototypes
    global function_prototypes
    for function_prototype in function_prototypes:
        NRPyLeakage_h_string += function_prototype+"\n"

    NRPyLeakage_h_string += "#endif // NRPYLEAKAGE_H_"

    # Step 4.h.iv: Write the file
    filename = os.path.join(Ccodesdir,"NRPyLeakage.h")
    with open(filename,"w") as file:
        file.write(NRPyLeakage_h_string)

    print("Wrote file",filename)

In [26]:
def write_param_ccl():
    with open(os.path.join(Thorndir,"param.ccl"),"w") as file:
        file.write("# Parameter definitions for thorn NRPyLeakageET")

def write_schedule_ccl():
    with open(os.path.join(Thorndir,"schedule.ccl"),"w") as file:
        file.write("# Schedule definitions for thorn NRPyLeakageET")

def write_interface_ccl():
    with open(os.path.join(Thorndir,"interface.ccl"),"w") as file:
        file.write(r"""# Interface definition for thorn NRPyLeakageET

implements: NRPyLeakageET
inherits:

includes header: NRPyLeakage.h in NRPyLeakage.h
USES INCLUDE HEADER: WVU_EOS_Tabulated_headers.h

# ------------------------------------------------------
void FUNCTION NRPyLeakage_compute_GRMHD_source_terms_and_opacities(CCTK_INT IN which_constants_to_use, \
                                                                   CCTK_REAL IN rho_b,                 \
                                                                   CCTK_REAL IN Y_e,                   \
                                                                   CCTK_REAL IN T,                     \
                                                                   CCTK_REAL IN ARRAY tau_nue,         \
                                                                   CCTK_REAL IN ARRAY tau_anue,        \
                                                                   CCTK_REAL IN ARRAY tau_nux,         \
                                                                   CCTK_REAL OUT R_source,             \
                                                                   CCTK_REAL OUT Q_source,             \
                                                                   CCTK_REAL OUT kappa_nue,            \
                                                                   CCTK_REAL OUT kappa_anue,           \
                                                                   CCTK_REAL OUT kappa_nux)

PROVIDES FUNCTION NRPyLeakage_compute_GRMHD_source_terms_and_opacities WITH NRPyLeakage_compute_GRMHD_source_terms_and_opacities_impl LANGUAGE C
# ------------------------------------------------------
void FUNCTION WVU_EOS_mue_mup_mun_muhat_Xn_and_Xp_from_rho_Ye_T( CCTK_REAL IN rho,    \
                                                                 CCTK_REAL IN Ye,     \
                                                                 CCTK_REAL IN T,      \
                                                                 CCTK_REAL OUT mu_e,  \
                                                                 CCTK_REAL OUT mu_p,  \
                                                                 CCTK_REAL OUT mu_n,  \
                                                                 CCTK_REAL OUT muhat, \
                                                                 CCTK_REAL OUT X_n,   \
                                                                 CCTK_REAL OUT X_p )
USES FUNCTION WVU_EOS_mue_mup_mun_muhat_Xn_and_Xp_from_rho_Ye_T
# ------------------------------------------------------
""")

def write_make_code_defn():
    indent = "       "
    string = "SRCS = "
    for i,filename in enumerate(make_code_defn_list):
        if i == 0:
            string += filename+" \\\n"
        else:
            string += indent+filename+" \\\n"
    string = string[:-2]
    with open(os.path.join(Ccodesdir,"make.code.defn"),"w") as file:
        file.write(string)

def write_ccl_files():
    write_param_ccl()
    write_schedule_ccl()
    write_interface_ccl()
    write_make_code_defn()

<a id='c_code_generation'></a>

# Step 9: C code generation \[Back to [Top](#toc)\]
$$\label{c_code_generation}$$

The function below can be used to generate all the C code implemented in this tutorial notebook.

In [27]:
def generate_NRPyLeakage_Ccode():
    Cfunc_NRPyLeakage_Fermi_Dirac_Integrals()
    Cfunc_NRPyLeakage_compute_GRMHD_source_terms_and_opacities_all_different_constants()
    Cfunc_NRPyLeakage_compute_GRMHD_source_terms_and_opacities()
    Cfunc_NRPyLeakage_compute_optical_depths()
    Cheader_NRPyLeakage_h()
    if generate_ET_thorn:
        write_ccl_files()

In [28]:
generate_NRPyLeakage_Ccode()

Output C function NRPyLeakage_Fermi_Dirac_integrals() to file NRPyLeakageET/src/NRPyLeakage_Fermi_Dirac_integrals.c
Output C function NRPyLeakage_compute_GRMHD_source_terms_and_opacities_nrpy_constants_impl() to file NRPyLeakageET/src/NRPyLeakage_compute_GRMHD_source_terms_and_opacities_nrpy_constants_impl.c
Output C function NRPyLeakage_compute_GRMHD_source_terms_and_opacities_harm_constants_impl() to file NRPyLeakageET/src/NRPyLeakage_compute_GRMHD_source_terms_and_opacities_harm_constants_impl.c
Output C function NRPyLeakage_compute_GRMHD_source_terms_and_opacities_impl() to file NRPyLeakageET/src/NRPyLeakage_compute_GRMHD_source_terms_and_opacities_impl.c
Output C function NRPyLeakage_compute_optical_depths_impl() to file NRPyLeakageET/src/NRPyLeakage_compute_optical_depths_impl.c
Wrote file NRPyLeakageET/src/NRPyLeakage.h
